In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

from collections import Counter

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.auto import tqdm
from IPython.display import display, update_display
import time
import re
import warnings
import json
warnings.filterwarnings('ignore')

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Устанавливаем тему по умолчанию
sb_dark = sns.dark_palette('skyblue', 8, reverse=True) # teal
sns.set(palette=sb_dark)

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score, max_error

In [5]:
path = ''
data = pd.read_csv('train_events.csv')
video = pd.read_csv('video_info_v2.csv')
targets = pd.read_csv('train_targets.csv')

In [6]:
data

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid
0,2024-06-01 06:40:58+03:00,Chelyabinsk,desktop,browser,Windows,Yandex Browser,1883,video_133074,10067243
1,2024-06-01 19:33:24+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,512,video_362960,10245341
2,2024-06-01 21:30:43+03:00,St.-Petersburg,desktop,browser,Windows,Chrome,5647,video_96775,10894333
3,2024-06-01 23:03:42+03:00,Moscow,smartphone,mobile app,Android,Rutube,1521,video_161610,10029092
4,2024-06-01 22:48:09+03:00,Moscow,smartphone,mobile app,Android,Rutube,71,video_116245,10452976
...,...,...,...,...,...,...,...,...,...
1759611,2024-06-30 17:08:36+03:00,Moscow,smartphone,mobile app,Android,Rutube,4230,video_162776,10026914
1759612,2024-06-30 01:20:16+03:00,Moscow,smartphone,browser,Android,Firefox Mobile,382,video_316157,10417567
1759613,2024-06-30 22:37:04+03:00,St.-Petersburg,desktop,browser,Windows,Microsoft Edge,1342,video_83304,10009094
1759614,2024-06-30 11:33:09+03:00,St.-Petersburg,smartphone,mobile app,Android,Rutube,801,video_132769,10574374


In [7]:
video

,rutube_video_id,title,category,duration,author_id
0,video_185549,Как собрать букет из мыльных тюльпанов - Силик...,Хобби,1559160,1015054
1,video_111035,"Осторожно, Киберземляне!, 1 сезон, 12 серия",Сериалы,1320007,1002180
2,video_476517,ПОПУЛЯРНЫЕ ВИДЕОИГРЫ в LEGO... перевод - TD BR...,Хобби,606145,1095337
3,video_157198,"Хороший лжец (фильм, 2019)",Фильмы,6577440,1043618
4,video_289824,Нашего старого гнобят по-всякому,Развлечения,859493,1009535
...,...,...,...,...,...
481475,video_196126,14. Landscape Layer Blend,Дизайн,462000,1121338
481476,video_390909,Рихард Штраус: Завтра! — Людмила Рубинская,Музыка,227628,1043510
481477,video_326364,"Поймай Тинипин! Королевство эмоций, 2 серия. С...",Мультфильмы,744576,1002799
481478,video_374957,ВОРКУТА. 1 СЕРИЯ. УЖАСЫ. ПСИХОЛОГИЧЕСКИЙ ТРИЛЛЕР,Люди и блоги,2597385,1008386


In [8]:
18/418

0.0430622009569378

In [9]:
targets

,viewer_uid,age,sex,age_class
0,10087154,30,male,1
1,10908708,25,female,1
2,10190464,34,male,2
3,10939673,25,male,1
4,10288257,48,male,3
...,...,...,...,...
180007,10237402,43,female,3
180008,10796936,32,male,2
180009,10684284,28,male,1
180010,10031093,50,female,3


In [10]:
data['ua_client_name'].nunique()

60

In [11]:
video['category'].nunique()

40

In [12]:
video['author_id'].nunique()

125442

In [13]:
(video['author_id'].value_counts() > 10).sum()

4700

In [14]:
(481480-473931)/481480

0.015678740549970923

In [15]:
video.nunique()

rutube_video_id    481480
title              473931
category               40
duration           263461
author_id          125442
dtype: int64

In [16]:
# объеденяем данные
df_video_sex = data[['rutube_video_id','viewer_uid']].merge(video[['rutube_video_id','title']],on='rutube_video_id').merge(targets[['viewer_uid','sex']],on='viewer_uid')
df_video_sex['sex'] = df_video_sex['sex'].replace({'female':-1, 'male':1})
df_video_sex = df_video_sex.groupby(['rutube_video_id', 'title'])['sex'].agg(['sum','count']).reset_index()


n=10 #минимальное кол-во просмотров у видео


#Выделяем датасет из видео по кол-вам просмотров
df_video_sex_1 = df_video_sex[df_video_sex['count'] >= n][((df_video_sex['count'] - df_video_sex['sum'].abs())/df_video_sex['count']) > 0.8]
df_video_sex_1['target_sex'] = 3
df_video_sex_2 = df_video_sex[df_video_sex['count'] >= n][((df_video_sex['count'] - df_video_sex['sum'].abs())/df_video_sex['count']) <= 0.8]
df_video_sex_2['target_sex'] = df_video_sex_2['sum'] > 0
df_video_sex_text = pd.concat([df_video_sex_2,df_video_sex_1])

# 0- видео которые смотрят чаще женщины
# 1- видео которые смотрят чаще мужчины
# 3- видео которые смотрят все


# Разделение данных на обучающую и тестовую выборки
X = df_video_sex_text['title']  # Тексты (названия)
y = df_video_sex_text['target_sex']  # Метки классов

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Преобразование текстов в TF-IDF представление
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(df_video_sex_text['title'])
X_train_tfidf = vectorizer.transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Инициализация и обучение модели логистической регрессии
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

# Предсказания на тестовой выборке
y_pred = model.predict(X_test_tfidf)

# Оценка модели
accuracy = accuracy_score(y_test, y_pred)

print(f'Размер датасета: {df_video_sex_text.shape}')
print(f'Accuracy: {accuracy:.4f}')
print(classification_report(y_test, y_pred))
model_sex = LogisticRegression(max_iter=1000)
X = vectorizer.transform(X)
model_sex.fit(X, y)

Размер датасета: (18968, 5)
Accuracy: 0.8100
              precision    recall  f1-score   support

           0       0.92      0.87      0.89      1777
           1       0.75      0.92      0.83      1335
           3       0.63      0.44      0.52       682

    accuracy                           0.81      3794
   macro avg       0.77      0.74      0.75      3794
weighted avg       0.81      0.81      0.80      3794



LogisticRegression(max_iter=1000)

In [17]:
#регресия по возрасту
from catboost import CatBoostRegressor

# объеденяем данные
df_video_age = data[['rutube_video_id','viewer_uid']].merge(video[['rutube_video_id','title']],on='rutube_video_id').merge(targets[['viewer_uid','age']],on='viewer_uid')
df_video_age = df_video_age.groupby(['rutube_video_id', 'title'])['age'].agg(['median','count']).reset_index()
df_video_age

n=10 #минимальное кол-во просмотров у видео


# Разделение данных на обучающую и тестовую выборки
X = df_video_age[df_video_age['count'] >= n]['title']  # Тексты (названия)
y = df_video_age[df_video_age['count'] >= n]['median']  # Метки классов

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Преобразование текстов в TF-IDF представление
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(df_video_sex_text['title'])
X_train_tfidf = vectorizer.transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Инициализация и обучение модели логистической регрессии
model = CatBoostRegressor(verbose=100)
model.fit(X_train_tfidf, y_train)

# Предсказания на тестовой выборке
y_pred = model.predict(X_test_tfidf)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
max_err = max_error(y_test, y_pred)
print()
print(f'Размер датасета: {X.shape}')
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')
print(f'Maximum Error: {max_err}')

model_age = CatBoostRegressor(verbose=100)
X = vectorizer.transform(X)
model_age.fit(X, y)

Learning rate set to 0.063647
0:	learn: 5.2385624	total: 60.4ms	remaining: 1m
100:	learn: 4.2797177	total: 1.26s	remaining: 11.2s
200:	learn: 4.0532879	total: 2.47s	remaining: 9.83s
300:	learn: 3.8933577	total: 3.7s	remaining: 8.59s
400:	learn: 3.7658178	total: 4.9s	remaining: 7.32s
500:	learn: 3.6750346	total: 6.22s	remaining: 6.2s
600:	learn: 3.5944939	total: 7.52s	remaining: 4.99s
700:	learn: 3.5239979	total: 8.82s	remaining: 3.76s
800:	learn: 3.4674389	total: 10.2s	remaining: 2.53s
900:	learn: 3.4129586	total: 11.5s	remaining: 1.26s
999:	learn: 3.3656971	total: 12.8s	remaining: 0us

Размер датасета: (20398,)
Mean Squared Error: 12.819922340508727
R-squared: 0.5346681604991604
Maximum Error: 17.473474266390532
Learning rate set to 0.065932
0:	learn: 5.2228769	total: 15.8ms	remaining: 15.7s
100:	learn: 4.2518612	total: 1.65s	remaining: 14.7s
200:	learn: 4.0191924	total: 3.26s	remaining: 13s
300:	learn: 3.8551729	total: 4.89s	remaining: 11.3s
400:	learn: 3.7373452	total: 6.57s	remaini

In [18]:
#регресия по возрасту
from catboost import CatBoostClassifier
import numpy as np
# объеденяем данные
df_video_age_class = data[['rutube_video_id','viewer_uid']].merge(video[['rutube_video_id','title']],on='rutube_video_id').merge(targets[['viewer_uid','age_class']],on='viewer_uid')
df_video_age_class = df_video_age_class.groupby(['rutube_video_id', 'title'])['age_class'].agg(['median','count']).reset_index()
df_video_age_class

n=10 #минимальное кол-во просмотров у видео




# Разделение данных на обучающую и тестовую выборки
X = df_video_age_class[df_video_age_class['count'] >= n]['title']  # Тексты (названия)

df_video_age_class.loc[df_video_age_class['count'] >= n, 'median'] = np.floor(df_video_age_class.loc[df_video_age_class['count'] >= n, 'median'])
y = df_video_age_class[df_video_age_class['count'] >= n]['median']  # Метки классов

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Преобразование текстов в TF-IDF представление
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(df_video_sex_text['title'])
X_train_tfidf = vectorizer.transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Инициализация и обучение модели логистической регрессии
model = CatBoostClassifier(verbose=100)
model.fit(X_train_tfidf, y_train)

# Предсказания на тестовой выборке
y_pred = model.predict(X_test_tfidf)

# Оценка модели
accuracy = accuracy_score(y_test, y_pred)

print(f'Размер датасета: {df_video_sex_text.shape}')
print(f'Accuracy: {accuracy:.4f}')
print(classification_report(y_test, y_pred))

model_age_class = CatBoostClassifier(verbose=100)
X = vectorizer.transform(X)
model_age_class.fit(X, y)

Learning rate set to 0.090899
0:	learn: 1.3094750	total: 42.7ms	remaining: 42.6s
100:	learn: 0.7206881	total: 4.96s	remaining: 44.2s
200:	learn: 0.6736087	total: 10s	remaining: 39.8s
300:	learn: 0.6481544	total: 15.1s	remaining: 35s
400:	learn: 0.6292631	total: 20.2s	remaining: 30.2s
500:	learn: 0.6168038	total: 25.4s	remaining: 25.3s
600:	learn: 0.6066902	total: 30.7s	remaining: 20.4s
700:	learn: 0.5992342	total: 36s	remaining: 15.3s
800:	learn: 0.5920765	total: 41.1s	remaining: 10.2s
900:	learn: 0.5855121	total: 46.3s	remaining: 5.08s
999:	learn: 0.5798886	total: 51.4s	remaining: 0us
Размер датасета: (18968, 5)
Accuracy: 0.7420
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         9
         1.0       0.83      0.58      0.68      1301
         2.0       0.71      0.93      0.80      2156
         3.0       0.78      0.22      0.34       328

    accuracy                           0.74      3794
   macro avg       0.58      0.43   

In [19]:
df_video = data[['rutube_video_id','viewer_uid']].merge(video,on='rutube_video_id').merge(targets[['viewer_uid','sex']],on='viewer_uid')

In [20]:
video[['author_id','category']].groupby(['author_id']).count().describe()

,category
count,125442.000000
mean,3.838268
std,54.433941
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,8603.000000


In [21]:
video[['author_id','category']].groupby(['category']).count().describe()

,author_id
count,40.00000
mean,12037.00000
std,23571.05078
min,484.00000
25%,2363.75000
50%,4894.00000
75%,10240.50000
max,141342.00000


In [22]:
targets['sex'].value_counts()

female    90098
male      89914
Name: sex, dtype: int64

In [23]:
df_video[['author_id','category', 'sex']].groupby(['sex'])['category'].count()

sex
female    1084515
male       675101
Name: category, dtype: int64

In [24]:
targets['age_class'].value_counts()

2    65600
1    63131
3    43690
0     7591
Name: age_class, dtype: int64

In [25]:
model_age_class
model_age
model_sex

LogisticRegression(max_iter=1000)

In [26]:
X_tfidf = vectorizer.transform(video['title'])

In [27]:
model_age_class.predict(X_tfidf).shape

(481480, 1)

In [28]:
video['age_class_vidio'] = model_age_class.predict(X_tfidf)
video['age_vidio'] = model_age.predict(X_tfidf)
video['sex_vidio'] = model_sex.predict(X_tfidf)
video

,rutube_video_id,title,category,duration,author_id,age_class_vidio,age_vidio,sex_vidio
0,video_185549,Как собрать букет из мыльных тюльпанов - Силик...,Хобби,1559160,1015054,2.0,35.948303,0
1,video_111035,"Осторожно, Киберземляне!, 1 сезон, 12 серия",Сериалы,1320007,1002180,2.0,31.002096,0
2,video_476517,ПОПУЛЯРНЫЕ ВИДЕОИГРЫ в LEGO... перевод - TD BR...,Хобби,606145,1095337,2.0,35.934286,1
3,video_157198,"Хороший лжец (фильм, 2019)",Фильмы,6577440,1043618,2.0,34.763324,1
4,video_289824,Нашего старого гнобят по-всякому,Развлечения,859493,1009535,2.0,35.934286,1
...,...,...,...,...,...,...,...,...
481475,video_196126,14. Landscape Layer Blend,Дизайн,462000,1121338,2.0,35.934286,1
481476,video_390909,Рихард Штраус: Завтра! — Людмила Рубинская,Музыка,227628,1043510,2.0,35.934286,1
481477,video_326364,"Поймай Тинипин! Королевство эмоций, 2 серия. С...",Мультфильмы,744576,1002799,2.0,35.007870,1
481478,video_374957,ВОРКУТА. 1 СЕРИЯ. УЖАСЫ. ПСИХОЛОГИЧЕСКИЙ ТРИЛЛЕР,Люди и блоги,2597385,1008386,2.0,33.824670,1


In [29]:
video.to_parquet('video_info_v2_sex_age.parquet', index=False)

In [30]:
pd.read_csv('test_viewer_uid.csv')

,Unnamed: 0,viewer_uid
0,0,10245341
1,1,10452976
2,2,10306496
3,3,10783507
4,4,10511293
...,...,...
175574,175574,10066164
175575,175575,10065963
175576,175576,10826550
175577,175577,10026914


In [31]:
pd.read_csv('all_events.csv')

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid
0,2024-06-01 13:08:30+03:00,Tatarstan Republic,smartphone,browser,Android,Chrome Mobile,60,video_395879,10813370
1,2024-06-01 14:30:00+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,60,video_216518,10512324
2,2024-06-01 18:48:12+03:00,Novosibirsk Oblast,smartphone,mobile app,Android,Rutube,121,video_41225,10951137
3,2024-06-01 16:32:36+03:00,Moscow,desktop,browser,Windows,Yandex Browser,2324,video_215886,10912434
4,2024-06-01 00:57:04+03:00,Moscow Oblast,smartphone,mobile app,Android,Rutube,6830,video_43631,10223585
...,...,...,...,...,...,...,...,...,...
8439619,2024-06-30 18:12:06+03:00,Nizhny Novgorod Oblast,smartphone,mobile app,Android,Rutube,1735,video_163270,10687999
8439620,2024-06-30 09:23:08+03:00,Chelyabinsk,smartphone,mobile app,Android,Rutube,4766,video_433627,10425804
8439621,2024-06-30 10:57:36+03:00,Krasnodar Krai,smartphone,mobile app,Android,Rutube,890,video_11911,10565147
8439622,2024-06-30 18:16:55+03:00,Moscow,smartphone,browser,Android,Yandex Browser,2504,video_252573,10715847
